In [1]:
# 📄 PDF 문서
#    ↓ (Upstage DocumentParser)
# 📜 전체 텍스트
#    ↓ (페이지 단위 청킹)
# 🧩 context 청크들
#    ↓ (GPT-4 API 호출, 커스텀 프롬프트로 질문/정답/이유 생성/CoT Reasoning)
# 📦 question / context / reason & answer /
#    ↓
# 💾 QA datasets csv 저장 (RAFT fine-tuning에 사용)

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## RAFT 학습용 QA 생성기

In [2]:
import re
import os
import requests
import json
from tqdm import tqdm
import openai
from langchain_core.documents import Document

In [7]:
# ✅ 파일 경로 설정
file_path = r"/Users/daeunbaek/nuebaek/BOAZ/BOAZ_ADV/Daeun/apm_guidelines/APM-11-055.pdf"
file_name = os.path.basename(file_path)
output_path = os.path.join(os.path.dirname(file_path), f"{os.path.splitext(file_name)[0]}.json")

In [4]:
import os
import requests
import json

# ✅ 기본 요청 설정
DEFAULT_CONFIG = {
    "ocr": False,
    "coordinates": True,
    "output_formats": "['html', 'text', 'markdown']", 
    "model": "document-parse",
    "base64_encoding": "['figure', 'chart', 'table']" 
}

# ✅ Upstage API 설정
api_url = "https://api.upstage.ai/v1/document-ai/document-parse"
api_key = os.environ.get("UPSTAGE_API_KEY")
headers = {"Authorization": f"Bearer {api_key}"}

# ✅ 분석 요청 실행
print(f"📤 파일 업로드 중: {file_name}")
with open(file_path, "rb") as pdf:
    response = requests.post(
        api_url,
        headers=headers,
        data=DEFAULT_CONFIG,
        files={"document": pdf}
    )

# ✅ 결과 처리 및 metadata 추가 저장
if response.status_code == 200:
    result = response.json()
    
    # 📎 메타데이터 구성
    metadata = {
        "document_name": file_name,
        "model": result.get("model"),
        "usage": result.get("usage"),
        "num_pages": len({elem["page"] for elem in result.get("elements", [])}),
    }

    # 결과와 메타데이터를 함께 저장
    result_with_metadata = {
        "metadata": metadata,
        "elements": result.get("elements", [])
    }

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(result_with_metadata, f, ensure_ascii=False, indent=2)
    
    print(f"✅ 분석 결과 및 메타데이터 저장 완료: {output_path}")
else:
    print(f"❌ 오류 발생 ({response.status_code}): {response.text}")


📤 파일 업로드 중: APM-11-055.pdf
✅ 분석 결과 및 메타데이터 저장 완료: /Users/daeunbaek/nuebaek/BOAZ/BOAZ_ADV/Daeun/apm_guidelines/APM-11-055.json


In [3]:
# ✅ 분석 결과 JSON 파일 경로
json_path = "/Users/daeunbaek/nuebaek/BOAZ/BOAZ_ADV/Daeun/apm_guidelines/APM-11-055.json"

In [4]:
# ✅ 분석 결과 JSON 파일 경로
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

elements = data["elements"]

# ✅ 페이지 필터링 + 문단 추출
# remove_pages = set(range(0, 5)) | set(range(63, 68))

filtered_elements = [
    elem for elem in elements
    if elem["category"] not in ("footer", "header", "footnote")
    # and elem["page"] not in remove_pages
    and elem["content"].get("markdown", "").strip()
]

In [5]:
# 제거할 페이지 필터링 + 문단 결합
from langchain_core.documents import Document

# remove_pages = set(range(0, 35)) | set(range(57, 69))  # 필요 없는 페이지 번호 제거
page_map = {}
for elem in elements:
    page = elem["page"]
    if elem["category"] in ("footer", "header", "footnote"):
        continue
    text = elem["content"].get("markdown", "").strip()
    if text:
        page_map[page] = page_map.get(page, "") + "\n" + text

In [ ]:
#LangChain Document 구성
documents = [
    Document(
        page_content=page_map[page].strip(),
        metadata={"page": page, "document_name": file_name}
    )
    for page in sorted(page_map.keys())
]

print(f"✅ 구성된 문서 수: {len(documents)}")
]

✅ 구성된 문서 수: 6


In [9]:
# split_docs는 페이지 단위로 그대로 사용
split_docs = documents
print(f"✅ QA 생성을 위한 페이지 청크 수: {len(split_docs)}")

✅ QA 생성을 위한 페이지 청크 수: 6


In [10]:
split_docs

[Document(metadata={'page': 1, 'document_name': 'APM-11-055.pdf'}, page_content='Anesth Pain Med 2016; 11: 55-63\nhttp://dx.doi.org/10.17085/apm.2016.11.1.55\nPharmacological and non-pharmacological interventions to\nalleviate anxiety before pediatric anesthesia: a survey of\ncurrent practice in Korea\nDepartment of Anesthesiology and Pain Medicine, Yeungnam University School of Medicine, *Kyungpook National University School of\nMedicine, Daegu, Korea\nHyo Eun Kang, Sung Mee Jung, and Sungsik Park*\nBackground: This study was undertaken to determine current\npractice for preoperative anxiety reduction in Korean children.\nMethods: An email survey of all members (n = 158) of the Korean\nSociety of Pediatric Anesthesiologists was conducted from\nNovember 2014 to January 2015 to assess current practice,\npreferences, and general opinions regarding pharmacological and\nnon-pharmacological interventions performed to alleviate preoperative\nanxiety in children prior to general anesthesia.\n

In [19]:
# prompt

# from langchain_core.prompts import PromptTemplate
# from langchain_openai import ChatOpenAI
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnableLambda
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# import json
# import difflib

# prompt = PromptTemplate.from_template(
#     """Context information is below. You are only aware of this context and nothing else.

# ### Context:
# ---------------------
# {context}
# ---------------------

# You are a medical expert in the field of {domain}. Based on the above clinical guideline context, your task is to generate exactly {num_questions} question(s) that a medical professional might ask to assess clinical understanding.

# ---

# Each Q&A pair must:
# - Present a **clinically plausible and distinct** question (no duplicates or variations of the same question)
# - Be grounded **explicitly in the given context** (no hallucinations)
# - Belong to one of the following clinical reasoning categories:
#   • Drug selection & dosing (e.g., appropriate drug, route, weight-based dosing)
#   • Contraindications & precautions
#   • Sedation monitoring & depth assessment
#   • Pre-sedation evaluation
#   • Recovery & discharge criteria
#   • Emergency management

# ---

# To add clinical realism, consider:
# - Comorbidities
# - Risk-benefit tradeoffs
# - Dosing adjustments or monitoring strategies

# ---

# Each entry must include:
# - `question`: A Korean clinical question
# - `reason`: A Korean explanation citing direct quotes using ##begin_quote## and ##end_quote##
# - `answer`: A Korean answer based strictly on the context

# To ensure quality:
# - Do not repeat medications, topics, or sentence structures
# - Vary the structure of your reasoning to avoid templated style

# Respond only in **valid JSON array format**. No markdown, no lists, no extra explanations.

# ---

# ### Example Format:
# [
#   {{
#     "question": "4살 환자의 적절한 endotracheal tube size와 depth는?",
#     "reason": "문서에 따르면 ##begin_quote## ETT 크기(직경, mm) = (나이/4) + 4 / 깊이(cm) = (나이/2) + 12 ##end_quote## 라고 명시되어 있습니다.",
#     "answer": "비커프 튜브는 5.0 mm, 커프 튜브는 4.5 mm, 깊이는 약 14 cm가 적절합니다."
#   }},
#   {{
#     "question": "소아 환자를 깨울 때 laryngospasm이 의심되면 어떻게 해야 하나요?",
#     "reason": "문서에서는 ##begin_quote## 후두경련 발생 시 양압환기, 하악 자극, 약물 투여, 기도 확보가 필요하다 ##end_quote## 라고 명시되어 있습니다.",
#     "answer": "산소 양압환기, 하악 자극, 프로포폴 또는 석시닐콜린 투여, 필요시 재삽관을 고려합니다."
#   }}
# ]
# """
# )


In [23]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Removed unnecessary import of convert_to_openai_image_block
from tqdm import tqdm
import difflib
import json
import re
import os
import pandas as pd

# ✅ 1. 프롬프트 템플릿 정의
medical_qa_prompt_template = PromptTemplate.from_template(medical_qa_prompt_template = """
# Role and Objective
You are a specialized medical expert in Pediatric Anesthesia.
Your primary task is to generate high-quality Korean Q&A data from provided medical guidelines for training a medical question-answering model.

# Core Principles
- Maintain strict medical accuracy and precision
- Focus on practical clinical scenarios
- Ensure questions are specific and actionable
- Include relevant numerical values when available
- Generate questions only from provided context
- Use clear and professional Korean language

# Input Data Structure
{
    "context": {context},
    "num_questions per page": {num_questions}
}

# Question Generation Guidelines
1. Content Adherence:
   - Generate questions strictly based on provided content
   - Include specific numerical values (doses, weights, ages, etc.)
   - Focus on practical clinical applications

2. Question Format:
   - Use open-ended, short-answer format
   - Write questions in Korean
   - Make questions specific and clinically relevant
   - Avoid multiple-choice formats

3. Reference Requirements:
   - Include all supporting sentences
   - For tables: include entire table content
   - For figures: include surrounding text and captions

# Answer Guidelines
- Start reasoning directly with a quote 
- Start with a step-by-step reasoning using quotes from the context
- Enclose all direct quotes in **##begin_quote## ... ##end_quote##**
- End with `<ANSWER>: ...` in Korean (full sentence) 

# Output Format
Return your output in this **strict JSON format**:
[
  {
    "question": "한국어로 된 의학 질문",
    "reference_sentences": [
      "관련 문장 1",
      "관련 문장 2"
    ],
    "answer": "##Reason: ...\n<ANSWER>: ..."
  }
]

# Quality Controls
- Ensure medical accuracy
- Verify all numerical values
- Check Korean language usage
- Validate JSON format
- Confirm reference alignment

# Error Prevention
- Double-check medical terminology
- Verify numerical calculations
- Ensure proper Korean grammar
- Validate JSON structure
- Confirm context alignment

# Example Outputs
Return as a **valid JSON array** of objects. Each object must look like:

[
  {{
    "question": "5살 23kg 소아 환자의 마취 중 적절한 수액 주입량은?",
    "reference_sentences": [
      "20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다.",
      "이 수액량은 전신마취 중 적절한 수분 공급을 위해 사용된다."
    ],
    "answer": "##Reason: 문서의 ##begin_quote## 20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다 ##end_quote## 라는 설명에 따르면, 23kg 소아는 1500mL + (3×20mL) = 1560mL가 필요합니다.\n<ANSWER>: 5살 23kg 소아의 마취 중 유지 수액량은 1560mL입니다."
  }},
  {{
    "question": "소아 환자를 깨울 때 laryngospasm이 의심되면 어떤 처치를 해야 하나요?",
    "reference_sentences": [
      "Laryngospasm이 의심되는 경우 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다.",
      "신속한 인식과 처치가 저산소증을 예방하는 데 중요하다."
    ],
    "answer": "##Reason: 문서에 따르면 ##begin_quote## 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다 ##end_quote## 라고 되어 있습니다. 이는 환자의 기도를 유지하고 저산소증을 방지하는 데 중요한 조치입니다.\n<ANSWER>: Laryngospasm이 의심되는 경우 jaw thrust, 양압 환기, succinylcholine 투여를 포함한 즉각적인 처치가 필요합니다."
  }}
]
"""
)

TypeError: PromptTemplate.from_template() missing 1 required positional argument: 'template'

In [20]:
# ✅ 2. LLM 설정
llm = ChatOpenAI(
    model="gpt-4.1",
    temperature=0.3,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [21]:
# ✅ 3. JSON 파서
def custom_json_parser(response):
    try:
        json_string = response.content.strip().removeprefix("```json\n").removesuffix("\n```").strip()
        json_string = f'[{json_string}]' if not json_string.startswith("[") else json_string
        return json.loads(json_string)
    except Exception as e:
        print(f"[PARSE ERROR] {e}")
        return []

In [22]:
# ✅ 4. 체인 구성
raft_chain = medical_qa_prompt_template | llm | RunnableLambda(custom_json_parser)

# ✅ 5. QA 생성
qa_dataset = []
question_set = set()
output_path = "qa_dataset.jsonl"
invalid_log_path = "qa_invalid_outputs.jsonl"

open(output_path, "w", encoding="utf-8").close()
open(invalid_log_path, "w", encoding="utf-8").close()

progress = tqdm(total=len(split_docs))
progress.set_description("QA 생성 중")

for doc_idx, doc in enumerate(split_docs):
    context = doc.page_content.strip()
    metadata = doc.metadata or {}
    page = metadata.get("page", -1)

    if not context or len(context) < 100:
        print(f"[SKIPPED: empty/short context] page={page}")
        progress.update(1)
        continue

    llm.temperature = 0.2 if doc_idx % 2 == 0 else 0.5

    try:
        results = raft_chain.invoke({
            "context": context,
            "num_questions": 4
        })
    except Exception as e:
        print(f"[ERROR: GPT 호출 실패] page={page}, error={e}")
        progress.update(1)
        continue

    valid_count = 0
    for qa in results:
        question = qa.get("question", "").strip()
        answer = qa.get("answer", "").strip()
        references = qa.get("reference_sentences", [])

        if not question or any(difflib.SequenceMatcher(None, question, q).ratio() > 0.85 for q in question_set):
            continue

        if not re.match(r"^##Reason: .*<ANSWER>: .*", answer, re.DOTALL):
            print(f"[SKIPPED: answer format invalid] {question}")
            with open(invalid_log_path, "a", encoding="utf-8") as f:
                json.dump({"issue": "invalid_answer_format", "question": question, "answer": answer}, f, ensure_ascii=False)
                f.write("\n")
            continue

        question_set.add(question)
        qa["metadata"] = metadata
        qa_dataset.append(qa)

        with open(output_path, "a", encoding="utf-8") as f:
            f.write(json.dumps(qa, ensure_ascii=False) + "\n")

        valid_count += 1
        if valid_count >= 5:
            break

    progress.update(1)

progress.close()
print(f"\n✅ 생성 완료! 총 {len(qa_dataset)}개의 QA 생성됨.")

# ✅ 엑셀로 저장
rows = []
for item in qa_dataset:
    rows.append({
        "page": item.get("metadata", {}).get("page"),
        "question": item.get("question"),
        "reference_sentences": "\n".join(item.get("reference_sentences", [])),
        "answer": item.get("answer"),
    })

df = pd.DataFrame(rows)
excel_output_path = r"/Users/daeunbaek/nuebaek/BOAZ/Daeun/raft_qa_dataset_1.xlsx"
df.to_excel(excel_output_path, index=False)
print(f"✅ 엑셀 파일 저장 완료: {excel_output_path}")

QA 생성 중:   0%|          | 0/6 [00:00<?, ?it/s]

[
  {
    "question": "한국 소아마취과 의사들이 소아의 수술 전 불안 완화를 위해 가장 많이 사용하는 약물은 무엇입니까?",
    "reference_sentences": [
      "Nearly half of the respondents (53.7%) used premedication to reduce anxiety, and midazolam was most frequently used.",
      "Anesthesiologists requiring effective anxiety reduction prefer pharmacological intervention and most commonly use intravenous midazolam,"
    ],
    "answer": "##Reason: 문맥에서 'Nearly half of the respondents (53.7%) used premedication to reduce anxiety, and midazolam was most frequently used.'와 'Anesthesiologists requiring effective anxiety reduction prefer pharmacological intervention and most commonly use intravenous midazolam,'라는 문장을 통해 소아마취과 의사들이 불안 완화를 위해 가장 많이 사용하는 약물이 미다졸람임을 알 수 있습니다.\n<ANSWER>: 한국 소아마취과 의사들은 소아의 수술 전 불안 완화를 위해 미다졸람을 가장 많이 사용합니다."
  },
  {
    "question": "한국 소아마취과 의사들이 비약물적 불안 완화 방법 중 가장 효과적이라고 생각하는 방법은 무엇이며, 실제로 몇 퍼센트가 이를 허용합니까?",
    "reference_sentences": [
      "Parental presence during induction of anesthesia was consi

QA 생성 중:  17%|█▋        | 1/6 [00:21<01:49, 21.88s/it]

[
  {
    "question": "설문에 참여한 소아마취과 의사들의 평균 임상 경력은 몇 년이며, 표준편차와 범위를 함께 설명하세요.",
    "reference_sentences": [
      "They had been in practice for 14.4 ± 8.4 years.",
      "Years in practice | Mean ± SD | 14.4 ± 8.4 | | | Range | 1–32 |"
    ],
    "answer": "##Reason: 설문에 참여한 소아마취과 의사들의 임상 경력에 대해 'They had been in practice for 14.4 ± 8.4 years.'와 'Years in practice | Mean ± SD | 14.4 ± 8.4 | | | Range | 1–32 |'에서 평균이 14.4년, 표준편차가 8.4년, 범위는 1년에서 32년임을 알 수 있습니다.\n<ANSWER>: 설문에 참여한 소아마취과 의사들의 평균 임상 경력은 14.4년이며, 표준편차는 8.4년이고, 경력 범위는 1년에서 32년입니다."
  },
  {
    "question": "설문에 응답한 소아마취과 의사들의 연령 분포를 구체적으로 설명하세요.",
    "reference_sentences": [
      "Respondents were typically in their forties (39%), ...",
      "| Age (yr) | 31–40 | 10 (24.4) | | | 41–50 | 16 (39.0) | | | 51–60 | 13 (31.7) | | | 61–70 | 2 (4.9) |"
    ],
    "answer": "##Reason: 'Respondents were typically in their forties (39%)'와 표의 연령 분포에서 31–40세가 24.4%(10명), 41–50세가 39.0%(16명), 51–60세가 31.7%(13명), 61–70세가 4.9%(2명)임을 확인할

QA 생성 중:  33%|███▎      | 2/6 [00:33<01:04, 16.06s/it]

...

[
  {
    "question": "한국에서 소아 마취 전 불안 완화를 위한 약물 전처치의 시행률은 얼마이며, 이는 다른 국가들과 어떻게 비교됩니까?",
    "reference_sentences": [
      "In Korea, the sedative premedication rate for pediatric anesthesia (54.7%), as determined by the present study, is similar to those reported in Turkey, Germany, and the United States [5,9,10], but is much higher than reported in the United Kingdom for day-case surgery (19%) [7]."
    ],
    "answer": "##Reason: 문맥에서는 한국의 소아 마취 전 진정제 전처치 시행률이 54.7%임을 명시하고 있으며, 이는 터키, 독일, 미국과 비슷하지만 영국의 당일 수술 시행률(19%)보다는 훨씬 높다고 설명하고 있습니다.\n<ANSWER>: 한국에서 소아 마취 전 진정제 전처치 시행률은 54.7%이며, 이는 터키, 독일, 미국과 비슷하지만 영국의 당일 수술 시행률(19%)보다는 훨씬 높습니다."
  },
  {
    "question": "한국에서 소아 마취 전 불안 완화에 가장 많이 사용되는 약물은 무엇이며, 투여 경로는 어떻게 됩니까?",
    "reference_sentences": [
      "Although numerous drugs have been used in clinical practice, midazolam is currently the mainstay of sedative premedication for pediatric anesthesia.",
      "Because oral or intranasal midazolam, commonly used as a premedication in 

QA 생성 중:  83%|████████▎ | 5/6 [01:31<00:18, 18.66s/it]

[
  {
    "question": "소아 마취에서 불안 감소를 위해 가장 선호되는 비약물적 중재 방법은 무엇입니까?",
    "reference_sentences": [
      "wanting safe reduction of anxiety preferred non-pharmacological intervention and most frequently used parental presence during induction the anesthesia."
    ],
    "answer": "##Reason: 문맥에서 'wanting safe reduction of anxiety preferred non-pharmacological intervention and most frequently used parental presence during induction the anesthesia.'라고 명시되어 있습니다. 이는 소아 마취에서 불안 감소를 위해 비약물적 중재가 선호되며, 그 중에서도 마취 유도 시 부모의 동반이 가장 많이 사용된다는 의미입니다.\n<ANSWER>: 소아 마취에서 불안 감소를 위해 가장 선호되는 비약물적 중재는 마취 유도 시 부모의 동반입니다."
  },
  {
    "question": "소아 마취에서 비약물적 불안 감소 중재로 부모 동반이 사용되는 시기는 언제입니까?",
    "reference_sentences": [
      "wanting safe reduction of anxiety preferred non-pharmacological intervention and most frequently used parental presence during induction the anesthesia."
    ],
    "answer": "##Reason: 문장에 따르면, 'parental presence during induction the anesthesia'라고 되어 있어, 부모의 동반이 마취 유도 시기에 사용된다는 것

QA 생성 중: 100%|██████████| 6/6 [01:46<00:00, 17.68s/it]


✅ 생성 완료! 총 24개의 QA 생성됨.
✅ 엑셀 파일 저장 완료: /Users/daeunbaek/nuebaek/BOAZ/Daeun/raft_qa_dataset_1.xlsx
